In [2]:
import sys
sys.path.append("..")
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import scipy.io
import os
import tarfile
import tqdm
import keras_utils

C:\Users\Banipreet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s
IMG_SIZE = 250

In [4]:
def decode_image_from_raw_bytes(raw_bytes):
    img = cv2.imdecode(np.asarray(bytearray(raw_bytes), dtype=np.uint8), 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [6]:
def image_center_crop(img):
    x_size, y_size = img.shape[0],img.shape[1]
    a = min(x_size,y_size)
    
    cropped_img = img[int((x_size/2)-(a/2)):int((x_size/2)+(a/2)),int((y_size/2)-(a/2)):int((y_size/2)+(a/2))]
    return cropped_img

In [9]:
def decode_image_from_raw_bytes(raw_bytes):
    img = cv2.imdecode(np.asarray(bytearray(raw_bytes), dtype=np.uint8), 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [16]:
def image_center_crop(img):
    x_size, y_size = img.shape[0],img.shape[1]
    a = min(x_size,y_size)
    
    cropped_img = img[int((x_size/2)-(a/2)):int((x_size/2)+(a/2)),int((y_size/2)-(a/2)):int((y_size/2)+(a/2))]
    return cropped_img

In [17]:
def prepare_raw_bytes_for_model(raw_bytes, normalize_for_model=True):
    img = decode_image_from_raw_bytes(raw_bytes)  # decode image raw bytes to matrix
    img = image_center_crop(img)  # take squared center crop
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # resize for our model
    if normalize_for_model:
        img = img.astype("float32")  # prepare for normalization
        img = keras.applications.inception_v3.preprocess_input(img)  # normalize for model
    return img

In [18]:
def read_raw_from_tar(tar_fn, fn):
    with tarfile.open(tar_fn) as f:
        m = f.getmember(fn)
        return f.extractfile(m).read()

In [22]:
def get_all_filenames(tar_fn):
    with tarfile.open(tar_fn) as f:
        return [m.name for m in f.getmembers() if m.isfile()]

all_files = sorted(get_all_filenames("102flowers.tgz"))  # list all files in tar sorted by name
all_labels = scipy.io.loadmat('imagelabels.mat')['labels'][0] - 1  # read class labels (0, 1, 2, ...)
# all_files and all_labels are aligned now
N_CLASSES = len(np.unique(all_labels))
print(N_CLASSES)

102


In [23]:
# split into train/test
tr_files, te_files, tr_labels, te_labels = \
    train_test_split(all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

In [24]:
def raw_generator_with_label_from_tar(tar_fn, files, labels):
    label_by_fn = dict(zip(files, labels))
    with tarfile.open(tar_fn) as f:
        while True:
            m = f.next()
            if m is None:
                break
            if m.name in label_by_fn:
                yield f.extractfile(m).read(), label_by_fn[m.name]

In [38]:
BATCH_SIZE = 32

def batch_generator(items, batch_size):
    batch = [None] * batch_size
    for i, item in enumerate(items):
        batch[i%batch_size] = item
        if i%batch_size == batch_size-1:
            yield batch
            batch = [None] * batch_size
    if batch[0]:
        yield [item for item in batch if item]

In [42]:
def train_generator(files, labels):
    while True:  # so that Keras can loop through this as long as it wants
        for batch in batch_generator(raw_generator_with_label_from_tar(
                "102flowers.tgz", files, labels), BATCH_SIZE):
            # prepare batch images
            batch_imgs = []
            batch_targets = []
            for raw, label in batch:
                img = prepare_raw_bytes_for_model(raw)
                batch_imgs.append(img)
                batch_targets.append(label)
            # stack images into 4D tensor [batch_size, img_size, img_size, 3]
            batch_imgs = np.stack(batch_imgs, axis=0)
            # convert targets into 2D tensor [batch_size, num_classes]
            batch_targets = keras.utils.np_utils.to_categorical(batch_targets, N_CLASSES)
            yield batch_imgs, batch_targets

In [44]:
s = reset_tf_session()

In [45]:
def inception(use_imagenet=True):
    # load pre-trained model graph, don't add final layer
    model = keras.applications.InceptionV3(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                          weights='imagenet' if use_imagenet else None)
    # add global pooling just like in InceptionV3
    new_output = keras.layers.GlobalAveragePooling2D()(model.output)
    # add new dense layer for our labels
    new_output = keras.layers.Dense(N_CLASSES, activation='softmax')(new_output)
    model = keras.engine.training.Model(model.inputs, new_output)
    return model

In [46]:
model = inception()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [47]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 250, 250, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 124, 124, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 124, 124, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 124, 124, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

____________________________________________________________________________________________________
conv2d_26 (Conv2D)               (None, 28, 28, 64)    18432       average_pooling2d_3[0][0]        
____________________________________________________________________________________________________
batch_normalization_20 (BatchNor (None, 28, 28, 64)    192         conv2d_20[0][0]                  
____________________________________________________________________________________________________
batch_normalization_22 (BatchNor (None, 28, 28, 64)    192         conv2d_22[0][0]                  
____________________________________________________________________________________________________
batch_normalization_25 (BatchNor (None, 28, 28, 96)    288         conv2d_25[0][0]                  
____________________________________________________________________________________________________
batch_normalization_26 (BatchNor (None, 28, 28, 64)    192         conv2d_26[0][0]         

____________________________________________________________________________________________________
conv2d_49 (Conv2D)               (None, 13, 13, 192)   215040      activation_48[0][0]              
____________________________________________________________________________________________________
conv2d_50 (Conv2D)               (None, 13, 13, 192)   147456      average_pooling2d_5[0][0]        
____________________________________________________________________________________________________
batch_normalization_41 (BatchNor (None, 13, 13, 192)   576         conv2d_41[0][0]                  
____________________________________________________________________________________________________
batch_normalization_44 (BatchNor (None, 13, 13, 192)   576         conv2d_44[0][0]                  
____________________________________________________________________________________________________
batch_normalization_49 (BatchNor (None, 13, 13, 192)   576         conv2d_49[0][0]         

batch_normalization_73 (BatchNor (None, 13, 13, 192)   576         conv2d_73[0][0]                  
____________________________________________________________________________________________________
activation_73 (Activation)       (None, 13, 13, 192)   0           batch_normalization_73[0][0]     
____________________________________________________________________________________________________
conv2d_74 (Conv2D)               (None, 13, 13, 192)   258048      activation_73[0][0]              
____________________________________________________________________________________________________
batch_normalization_74 (BatchNor (None, 13, 13, 192)   576         conv2d_74[0][0]                  
____________________________________________________________________________________________________
activation_74 (Activation)       (None, 13, 13, 192)   0           batch_normalization_74[0][0]     
___________________________________________________________________________________________

                                                                   activation_89[0][0]              
____________________________________________________________________________________________________
concatenate_2 (Concatenate)      (None, 6, 6, 768)     0           activation_92[0][0]              
                                                                   activation_93[0][0]              
____________________________________________________________________________________________________
activation_94 (Activation)       (None, 6, 6, 192)     0           batch_normalization_94[0][0]     
____________________________________________________________________________________________________
mixed10 (Concatenate)            (None, 6, 6, 2048)    0           activation_86[0][0]              
                                                                   mixed9_1[0][0]                   
                                                                   concatenate_2[0][0]     

In [49]:
for layer in model.layers:
    layer.trainable = True
    
for layer in model.layers[:-50]:
    layer.trainable = False

In [50]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.adamax(lr=1e-2),
    metrics=['accuracy']
)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [52]:
model.fit_generator(
    train_generator(tr_files, tr_labels), 
    steps_per_epoch=len(tr_files) // BATCH_SIZE // 8,
    epochs=2 * 8,
    validation_data=train_generator(te_files, te_labels), 
    validation_steps=len(te_files) // BATCH_SIZE // 4,
    callbacks=[keras_utils.TqdmProgressCallback(), 
               keras_utils.ModelSaveCallback(model_filename)],
    verbose=0,
    initial_epoch=last_finished_epoch or 0
)

Epoch 1/16


HBox(children=(IntProgress(value=0, max=25), HTML(value='')))


Model saved in flowers.000.hdf5
Epoch 2/16


HBox(children=(IntProgress(value=0, max=25), HTML(value='')))


Model saved in flowers.001.hdf5
Epoch 3/16


...


Model saved in flowers.013.hdf5
Epoch 15/16


HBox(children=(IntProgress(value=0, max=25), HTML(value='')))


Model saved in flowers.014.hdf5
Epoch 16/16


HBox(children=(IntProgress(value=0, max=25), HTML(value='')))


Model saved in flowers.015.hdf5


In [53]:
test_accuracy = model.evaluate_generator(
    train_generator(te_files, te_labels), 
    len(te_files) // BATCH_SIZE // 2
)[1]
print(test_accuracy)

0.93625
